In [9]:
import numpy as np

In [10]:
def buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon):
    # Iterate through the sellers from strongest to weakest to find min price they'd accept
    for seller_index in range(seller_product_values.size):
        try:
            seller_current_partner = np.nanargmax(pairings[:, seller_index])
            seller_current_price = pairings[seller_current_partner, seller_index]
            min_price = seller_current_price+epsilon
        except ValueError:
            seller_current_partner = seller_current_price = np.nan
            min_price = seller_product_values[seller_index]

        # Buyers from strongest to weakest make offer to seller if:
        # - their current price is more than min_price, or
        # - their current price is NaN and they can afford min_price
        for buyer_index in range(buyer_product_values.size):
            try:
                buyer_current_partner = np.nanargmax(pairings[buyer_index, :])
                buyer_current_price = pairings[buyer_index, buyer_current_partner]
            except ValueError:
                buyer_current_partner = buyer_current_price = np.nan
            if not np.isnan(buyer_current_price) and buyer_current_price <= min_price: continue
            elif np.isnan(buyer_current_price) and buyer_product_values[buyer_index] <= min_price: continue
            # seller accepts offer, break
            if not np.isnan(seller_current_partner): pairings[seller_current_partner, seller_index] = np.nan
            if not np.isnan(buyer_current_partner): pairings[buyer_index, buyer_current_partner] = np.nan
            pairings[buyer_index, seller_index] = min_price
            return

In [16]:
def interpret_pairings(buyer_product_values, seller_product_values, pairings):
    empty_output = True
    for buyer_index in range(buyer_product_values.size):
        try:
            buyer_current_partner = np.nanargmax(pairings[buyer_index, :])
            buyer_current_price = pairings[buyer_index, buyer_current_partner]
            print(f"Buyer {buyer_index} buys from Seller {buyer_current_partner} for {buyer_current_price}")
            empty_output = False
        except ValueError:
            continue
    if empty_output: print("No deals were made.")

In [17]:
buyer_product_values = np.sort(np.array([2,4,5]))[::-1] # the max prices buyers are willing to pay, sorted from strong to weak
seller_product_values = np.sort(np.array([0,1,3])) # the min prices sellers are willing to accept, sorted from strong to weak
epsilon = 0.1
trials = 1000 # does 1000 of each buyer seller

# matrix of successful deals
# buyer i sells to seller j at price k
pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
pairings[:] = np.nan
# pairings[0,0]=2.5
# pairings[1,1]=2.5

# Run 1000 rounds of deals
print("Starting pairings:")
interpret_pairings(buyer_product_values, seller_product_values, pairings)
print("...")

for round in range(trials):
    previous_pairings = pairings.copy()
    buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
    interpret_pairings(buyer_product_values, seller_product_values, pairings)
    print()
    if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break

print(f"Took {round+1} rounds to reach steady state:")
interpret_pairings(buyer_product_values, seller_product_values, pairings)

Starting pairings:
No deals were made.
...
Buyer 0 buys from Seller 0 for 0.0

Buyer 1 buys from Seller 0 for 0.1

Buyer 0 buys from Seller 0 for 0.2

Buyer 1 buys from Seller 0 for 0.30000000000000004

Buyer 0 buys from Seller 0 for 0.4

Buyer 1 buys from Seller 0 for 0.5

Buyer 0 buys from Seller 0 for 0.6

Buyer 1 buys from Seller 0 for 0.7

Buyer 0 buys from Seller 0 for 0.7999999999999999

Buyer 1 buys from Seller 0 for 0.8999999999999999

Buyer 0 buys from Seller 0 for 0.9999999999999999

Buyer 1 buys from Seller 0 for 1.0999999999999999

Buyer 0 buys from Seller 0 for 1.2

Buyer 1 buys from Seller 0 for 1.3

Buyer 0 buys from Seller 0 for 1.4000000000000001

Buyer 1 buys from Seller 0 for 1.5000000000000002

Buyer 0 buys from Seller 0 for 1.6000000000000003

Buyer 1 buys from Seller 0 for 1.7000000000000004

Buyer 0 buys from Seller 0 for 1.8000000000000005

Buyer 1 buys from Seller 0 for 1.9000000000000006

Buyer 0 buys from Seller 0 for 2.0000000000000004

Buyer 1 buys from Se

In [12]:
buyer_product_values = np.sort(np.random.random(np.random.randint(1,5)))[::-1] # the max prices buyers are willing to pay, sorted from strong to weak
seller_product_values = np.sort(np.random.random(np.random.randint(1,5))) # the min prices sellers are willing to accept, sorted from strong to weak
precision = 2
epsilon = 10**-precision

# matrix of successful deals
# buyer i sells to seller j at price k
pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
pairings[:] = np.nan
# randomly generate pairings and random prices
# buyer_permutation = np.random.permutation(np.arange(buyer_product_values.size))
# seller_permutation = np.random.permutation(np.arange(seller_product_values.size))
# for i in range(np.min([buyer_product_values.size, seller_product_values.size])):
#     if buyer_product_values[buyer_permutation[i]] >= seller_product_values[seller_permutation[i]]:
#         pairings[buyer_permutation[i], seller_permutation[i]] = np.random.uniform(
#             buyer_product_values[buyer_permutation[i]],
#             seller_product_values[buyer_permutation[i]]
#         )

# Run 1000 rounds of deals
print("buyer_product_values", buyer_product_values)
print("seller_product_values", seller_product_values)
print("Starting pairings:")
print(pairings)
print("...")

trials = 1000
total_rounds = 1
for round in range(trials):
    previous_pairings = pairings.copy()
    buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
    if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break
total_rounds += round

print(f"Took {total_rounds+round+1} rounds to reach steady state:")
print(pairings)

buyer_product_values [0.81669729]
seller_product_values [0.38149964]
Starting pairings:
[[nan]]
...
Took 4 rounds to reach steady state:
[[0.38149964]]
